<H1>
Custom Model
</H1>

In [4]:
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets

# Step 1: Data Loading and Preprocessing
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = r'Data\archive\Master Folder'
image_datasets = {x: datasets.ImageFolder(root=data_dir, transform=data_transforms[x]) for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

# Step 2: Define a Custom CNN Model
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CustomCNN(len(class_names))

# Step 3: Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Step 4: Training Loop
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(100):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(dataloaders['train'])}")

print("Finished Training")

# Step 5: Evaluate the Model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on test data: {accuracy:.2f}%")


Epoch 1, Loss: 0.3753288185326071
Epoch 2, Loss: 0.3296852434862433
Epoch 3, Loss: 0.32172863615060493
Epoch 4, Loss: 0.3209098515786286
Epoch 5, Loss: 0.3196634255751247
Epoch 6, Loss: 0.30535163847750335
Epoch 7, Loss: 0.30697402130937046
Epoch 8, Loss: 0.3083557003769733
Epoch 9, Loss: 0.3021257318674022
Epoch 10, Loss: 0.30161121252190226
Epoch 11, Loss: 0.3002430551224925
Epoch 12, Loss: 0.30799874533002425
Epoch 13, Loss: 0.3021688620771063
Epoch 14, Loss: 0.29760451576080466
Epoch 15, Loss: 0.30770622203853604
Epoch 16, Loss: 0.3027622269875055
Epoch 17, Loss: 0.30312761006232863
Epoch 18, Loss: 0.30098555444092334
Epoch 19, Loss: 0.2897601078254029
Epoch 20, Loss: 0.2929577011144183
Epoch 21, Loss: 0.2878304340817782
Epoch 22, Loss: 0.28942535006091497
Epoch 23, Loss: 0.29199744299137237
Epoch 24, Loss: 0.28504278804400596
Epoch 25, Loss: 0.2828099105894898
Epoch 26, Loss: 0.2915983877688314
Epoch 27, Loss: 0.29363576993588564
Epoch 28, Loss: 0.292184005094152
Epoch 29, Loss: 0

<H1>
Efficient Net
</H1>

In [6]:
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import timm

# Step 1: Data Loading and Preprocessing
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = r'Data\archive\Master Folder'
image_datasets = {x: datasets.ImageFolder(root=data_dir, transform=data_transforms[x]) for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

model_name = "efficientnet_b0"  # You can choose from different versions (e.g., efficientnet_b0, efficientnet_b1, etc.)
model = timm.create_model(model_name, pretrained=True)
num_features = model.classifier.in_features
model.classifier = nn.Linear(num_features, len(class_names))

model = CustomCNN(len(class_names))

# Step 3: Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Step 4: Training Loop
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(100):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(dataloaders['train'])}")

print("Finished Training")

# Step 5: Evaluate the Model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on test data: {accuracy:.2f}%")


c:\Users\abbas\.conda\envs\Uni\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\abbas\.conda\envs\Uni\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\abbas\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https:/

Epoch 1, Loss: 0.35642151709959313
Epoch 2, Loss: 0.32512097746708524
Epoch 3, Loss: 0.3245386958613881
Epoch 4, Loss: 0.3089955276948119
Epoch 5, Loss: 0.3120388691503984
Epoch 6, Loss: 0.32582573278298166
Epoch 7, Loss: 0.3046195779868218
Epoch 8, Loss: 0.3094801496451229
Epoch 9, Loss: 0.30638638262024154
Epoch 10, Loss: 0.2987865160243606
Epoch 11, Loss: 0.30875881570200936
Epoch 12, Loss: 0.30699448909291993
Epoch 13, Loss: 0.29874549828968305
Epoch 14, Loss: 0.29450577959335206
Epoch 15, Loss: 0.2969128592736105
Epoch 16, Loss: 0.2925247289690442
Epoch 17, Loss: 0.2927436823073816
Epoch 18, Loss: 0.2952354799568321
Epoch 19, Loss: 0.2882362373699486
Epoch 20, Loss: 0.2879624851740426
Epoch 21, Loss: 0.29062120628207366
Epoch 22, Loss: 0.29599023367309435
Epoch 23, Loss: 0.2872335499699563
Epoch 24, Loss: 0.29192058513889635
Epoch 25, Loss: 0.2867845134137953
Epoch 26, Loss: 0.28595708265249725
Epoch 27, Loss: 0.2868308588571597
Epoch 28, Loss: 0.2827350831275536
Epoch 29, Loss: 0

<H1>
Vision Transformer
</H1>

In [9]:
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import timm

# Step 1: Data Loading and Preprocessing
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = r'Data\archive\Master Folder'
image_datasets = {x: datasets.ImageFolder(root=data_dir, transform=data_transforms[x]) for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

# Step 2: Load a Pre-trained Vision Transformer (ViT) Model
model_name = "vit_base_patch16_224"  # You can choose from different ViT models
model = timm.create_model(model_name, pretrained=True)
num_features = model.head.in_features
model.head = nn.Linear(num_features, len(class_names))


# Step 3: Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Step 4: Training Loop
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(10):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(dataloaders['train'])}")

print("Finished Training")

# Step 5: Evaluate the Model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on test data: {accuracy:.2f}%")


Epoch 1, Loss: 0.3712931394370647
Epoch 2, Loss: 0.30862404140868815
Epoch 3, Loss: 0.31355870294568156
Epoch 4, Loss: 0.3170904924947644
Epoch 5, Loss: 0.29737916739041254
Epoch 6, Loss: 0.3037817850639718
Epoch 7, Loss: 0.2943939291196962
Epoch 8, Loss: 0.297940052509114
Epoch 9, Loss: 0.29025053170589266
Epoch 10, Loss: 0.2951931193761777
Finished Training
